In [1]:
!python -V

Python 3.8.10


In [ ]:
!pip install -U scikit-learn
!pip install pandas
!pip install pyarrow
!pip install fastparquet

In [3]:
import pandas as pd

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error
from math import sqrt

In [5]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    return df

In [6]:
# Q1
df_train = read_dataframe('./data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('./data/yellow_tripdata_2023-02.parquet')

num_columns = df_train.shape[1]
print(num_columns)

19


In [14]:
df_train['duration'] = df_train.tpep_dropoff_datetime - df_train.tpep_pickup_datetime
df_train.duration = df_train.duration.apply(lambda td: td.total_seconds() / 60)

df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)

In [8]:
# Q2
df_train.duration.std()

42.59435124195458

In [9]:
# Q3
((df_train.duration >= 1) & (df_train.duration <= 60)).mean()

0.9812202822125979

In [10]:
# Q4
categorical = ['PULocationID', 'DOLocationID']
df_train[categorical] = df_train[categorical].astype('str')
train_dicts = df_train[categorical].to_dict(orient='records')

dv = DictVectorizer()

X_train = dv.fit_transform(train_dicts)

print(X_train.shape)

(3066766, 518)


In [11]:
# Q5
target = 'duration'
y_train = df_train[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)
mse = mean_squared_error(y_train, y_pred, squared=False)
print(sqrt(mse))

6.480470485970359


In [16]:
# Q6
df_val[categorical] = df_val[categorical].astype('str')
val_dicts = df_val[categorical].to_dict(orient='records')

X_val = dv.transform(val_dicts)

y_val = df_val[target].values

lr.fit(X_val, y_val)

y_pred = lr.predict(X_val)
mse = mean_squared_error(y_val, y_pred, squared=False)
print(sqrt(mse))

6.497695020706183
